<a href="https://colab.research.google.com/github/azzbc7819/test_igaimers/blob/develop/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%B2%98%EB%A6%AC_withPytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader


In [65]:
#데이터 전처리

trainfile = '/content/drive/MyDrive/LG Aimers 해커톤/train.csv'
data = pd.read_csv(trainfile)

data_x = data.drop(columns=['PRODUCT_ID', 'TIMESTAMP','Y_Class', 'Y_Quality']) 
data_y = pd.DataFrame(data['Y_Class']) 

data_x = data_x.fillna(0) #결측값 모두 0으로 채우기

# qualitative to quantitative
#말그대로 범주형 데이터 > 수치형 데이터로 전환

qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(data_x[i])
    data_x[i] = le.transform(data_x[i])
    
print('Done.')

Done.


In [70]:
class CustomDataset(Dataset):
  def __init__(self, x_data, y_data):
    self.x_data = torch.Tensor(x_data)
    self.y_data = torch.Tensor(y_data)

  def __len__(self):
    # 가지고 있는 데이터셋의 길이를 반환한다.
    return len(self.x_data)

  def __getitem__(self,idx):
    return self.x_data[idx], self.y_data[idx] # 해당하는 idx(인덱스)의 x와 y 데이터를 반환한다.

In [68]:
print(type(data_x.values))
print(type(data_x))

#그냥 data는 pd.df라서 torch에서 못 읽음 values로 해주어야 함

<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


In [71]:
#DataLoader을 통해 DataSet으로 전처리하 데이터셋의 샘플들을 미니배치로 전달하고, 매 에폭마다 데이터를 다시 섞어준다
dataset = CustomDataset(data_x.values, data_y.values)
dataloader = DataLoader(dataset, batch_size = 60, shuffle=True)

In [74]:
train_x, train_y = next(iter(dataloader))

In [91]:
#batchsize, feature 수
print(train_x.size())
print(train_y.size())

torch.Size([60, 2877])
torch.Size([60, 1])


In [93]:
print(train_x[0])

tensor([5., 2., 2.,  ..., 0., 0., 0.])


In [94]:
train_x, train_y = next(iter(dataloader))

In [95]:
print(train_x[0])

#위랑 달라진 걸 볼 수 있음 next했으니까 다음 배치로 넘어간 것

tensor([4., 2., 2.,  ..., 0., 0., 0.])
